In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import sys
import hashlib
from PIL import Image

CURRENT_PATH = os.getcwd()

**Файл CIB стройки**

In [46]:
df1 = pd.read_excel('data/CIB стройки.xlsx', sheet_name = 'Лист1', skiprows = 4, dtypes = str)
df1 = df1.drop_duplicates()
df1.rename(columns = {'Unnamed: 0':'coordinates', 'Unnamed: 27':'coordinates_add'}, inplace = True)
df1['coordinates'].fillna(df1['coordinates_add'], inplace = True)
df1['coordinates'] = df1['coordinates'].astype(str)

df1['latitude'] = df1['coordinates'].apply(lambda x: x.split(',')[0] if x != 'nan' else np.nan )
df1['longitude'] = df1['coordinates'].apply(lambda x: x.split(',')[1] if x != 'nan' else np.nan)

df1 = df1.loc[(~df1['latitude'].isnull()) & (~df1['longitude'].isnull())]

df1 = df1.groupby(['latitude', 'longitude'])['Наименование проекта'].max().reset_index()

df1['coordinates'] = df1['latitude'] + ',' + df1['longitude']
df1.drop(['latitude', 'longitude'], axis = 1, inplace = True)
df1.rename(columns = {'Наименование проекта':'name'}, inplace = True)
df1.head()

,name,coordinates
0,6-ти секционный 17-ти этажный жилой дом (включ...,"54.883001, 38.056651"
1,ЖК Time,"54.886880, 38.078390"
2,"финансирование строительства ""Жилые дома в Окт...","55.354277, 37.539131"
3,финансирования / возмещения ранее понесенных и...,"55.403809, 37.536859"
4,строительства объектов жилой недвижимости «ЖК ...,"55.422360, 37.760512"


**Файл Геокодирование стройки Москва и МО.xlsx**

In [45]:
df2 = pd.read_excel('data/Геокодирование стройки Москва и МО.xlsx', sheet_name = 'Лист1', dtypes = str)
df2.rename(columns = {'Широта':'latitude', 'Долгота':'longitude'}, inplace = True)
df2 = df2.loc[(~df2['latitude'].isnull()) & (~df2['longitude'].isnull())]
df2 = df2.groupby(['latitude', 'longitude'])['Наименование проекта'].max().reset_index()

df2['coordinates'] = df2['latitude'].astype(str) + ',' + df2['longitude'].astype(str)
df2.drop(['latitude', 'longitude'], axis = 1, inplace = True)

df2.rename(columns = {'Наименование проекта':'name'}, inplace = True)
df2.head()

,name,coordinates
0,6-ти секционный 17-ти этажный жилой дом (включ...,"54.883001,38.056651"
1,I этап II очереди - 6-ти секционный 17-ти этаж...,"54.886880000000005,38.07839"
2,ЖК Эстет,"55.354277,37.539131"
3,финансирование стр-ва ж/д,"55.403808999999995,37.536859"
4,"ЖК ""Лунный""","55.42236,37.760512"


**Файл data/жк.xlsx**

In [43]:
df3 = pd.read_excel('data/жк.xlsx', sheet_name = 'Лист1', dtype = str)
df3 = df3.drop_duplicates()
df3 = df3.loc[(~df3['latitude'].isnull()) & (~df3['longitude'].isnull())]
df3 = df3.groupby(['latitude', 'longitude'])['name'].max().reset_index()
df3['coordinates'] = df3['latitude'].astype(str) + ',' + df3['longitude'].astype(str)
df3.drop(['latitude', 'longitude'], axis = 1, inplace = True)
df3.head()

,name,coordinates
0,"ЖК ""Гагаринские Высоты""","0,0"
1,ЖД по ул.Автодорожная,"129.68619,62.0058"
2,Многоквартирный жилой дом с соцкульбытом по ул...,"129.71612,62.03354"
3,"Жилой Комплекс ""Южный""","142.74195,46.96819"
4,"ЖК ""Солнце""","29.09225,59.90423"


**Объединим данные**

In [143]:
#df_all = pd.concat([df1, df2, df3])
df_all = df1.copy()
df_all = df_all.groupby('coordinates')['name'].max().reset_index()
df_all = df_all.loc[~df_all['coordinates'].isin(['0,0'])]
df_all.head()

,coordinates,name
0,"54.883001, 38.056651",6-ти секционный 17-ти этажный жилой дом (включ...
1,"54.886880, 38.078390",ЖК Time
2,"55.354277, 37.539131","финансирование строительства ""Жилые дома в Окт..."
3,"55.403809, 37.536859",финансирования / возмещения ранее понесенных и...
4,"55.422360, 37.760512",строительства объектов жилой недвижимости «ЖК ...


In [144]:
def check_coordinates(coord):
    lat, lon = coord.split(',')[0], coord.split(',')[1]
    try:
        lat, lon = float(lat), float(lon)
        if np.isnan(lat) or np.isnan(lon):
            return False
        if lat > 90:return False
        if lon > 180:return False
        return True
    except:
        return False
df_all['is_correct'] = df_all['coordinates'].apply(lambda x: check_coordinates(x))

In [145]:
df_all.head()

,coordinates,name,is_correct
0,"54.883001, 38.056651",6-ти секционный 17-ти этажный жилой дом (включ...,True
1,"54.886880, 38.078390",ЖК Time,True
2,"55.354277, 37.539131","финансирование строительства ""Жилые дома в Окт...",True
3,"55.403809, 37.536859",финансирования / возмещения ранее понесенных и...,True
4,"55.422360, 37.760512",строительства объектов жилой недвижимости «ЖК ...,True


In [146]:
df_all.loc[df_all['is_correct'] == True]

,coordinates,name,is_correct
0,"54.883001, 38.056651",6-ти секционный 17-ти этажный жилой дом (включ...,True
1,"54.886880, 38.078390",ЖК Time,True
2,"55.354277, 37.539131","финансирование строительства ""Жилые дома в Окт...",True
3,"55.403809, 37.536859",финансирования / возмещения ранее понесенных и...,True
4,"55.422360, 37.760512",строительства объектов жилой недвижимости «ЖК ...,True
5,"55.471071, 37.545797",финансирование стр-ва ж/д,True
6,"55.539400, 37.148104",состав жилого комплекса из трех 16-этажных дом...,True
7,"55.542003, 37.085383",финансирование и возмещения ранее понесенных з...,True
8,"55.546325, 37.304680",группа многоэтажных жилых домов со встроенными...,True
9,"55.569204, 38.211835","Финансирование, в т.ч. возмещение ранее понесе...",True


In [147]:
df_all.loc[df_all['is_correct'] == True][['coordinates', 'name']].to_csv('data/all_coordinates.csv', index = False)

In [148]:
df_all.loc[df_all['is_correct'] == True].shape

(84, 3)